# TMT Phase 3 Behaviour guide

Phase 3 covers cloning of TMT configurations. As defined on [Trello](https://trello.com/b/UicisECD/tmt-editor-features):

![](images/Phase3TrelloCards.gif)

## Cloning Rules

We will need to define what we expect to happen when we "clone" a TMT configuration. What we generally mean by "Clone" is to *take an existing TMT **Pattern**, and replicate this as something new*.  

We can help to define what a pattern is, by defining some rules about TMT that we want to ensure are not broken when we perform a cloning action:

+ TMT configurations are unique for each **Tool Run** - so we must create something new by tweaking at least one part of the Tool's critera so it does not clash with the original.
    + Hence **Clone by Stream** and **Clone by Economy** cases - we are referring to the part of the criteria that will change.
+ Model Outputs must only come from one source - so the model output of a clone must also have a new criteria
    + Keeping this as simple as possible, the best way to achieve this is to change the Stream to match the stream of the tool run.
+ For *Economy* clones, we want to extend the changes to any matching **input** model that is also specific to the economy
    + i.e. if the tool takes a *USD* YC as an input, clones to other economies should use their own economies yield curve.
    + Outputs will almost certainly be economy-specific, so these should be matching too.
    + Note that *Economy* behaviour will also be required for *Equity Asset* and *SubModel* behavior too.
        + See earlier suggestions that a single Tuple of *(Economy, Asset, SubModel)* is how we should best treat these 3 fields as one
    + Any "economy"-switching should also apply to the *EconAssetOverride* and *SubModelNameOverride*, if these are set to be economy-specific, rather than the native field. 
+ We want cloning to be **predictable**  - so keeping enforced changes to a minimum.
    + Limiting model changes to what we need to do - i.e. to meet the rules above - is safer than having more complex rules.
    + We can always edit calibrations in more detail, using the phase 2 functionality, to get exactly what we want in each specific case.
    + In summary - we will not edit the input models any more than we have to - they will retain their original *Stream*, *CalibrationType* and *CurveType* when performing a full clone.
    
## A BIG QUESTION

**Do we want to allow changes to be made to existing TMT configurations?**

Do we want to make this an explicit option - i.e. include a "don't change existing TMT" flag to act as a safety catch?

For "CLONE", the intention is to create new content - new tool runs and models.

For "APPLY, the intention is to make an edit to an existing config, and broadcast this change to other existing config. So this is always a change to existing.

There is plenty grey space inbetween these two intended use cases, and I'm not sure if we need to implement something else in addition to the "cloning rules".  

## Reviewing the TMT editor display

Taking into account what we want to do in this phase, I have mocked up what I feel we need to include to access this functionality.  

*Caveat* - I have seen the work in progress here, so I am being influenced by what has already been implemented so far.  

![](images/TMTEditor_Cloning_Mockup.png)

Comparing this with what we currently have, the suggested changes are:

+ Separate sections for **Editing Current Config** and **Cloning Config**
+ No hidden buttons - all options should be visible
+ "Remove" becomes a **Mark for Remove** - this should not happen immediately
    + and I guess we will need to indicate in the table what is to be removed
    + or alternatively, have some kind of "delete" toggle in the table, then there is no need for this button.
+ There is only one **"Commit Changes"** button
    + This will combine all of the <span style="color:orange">orange</span> buttons on the current editor into a single commit.
    + The positioning of this button is a bit fishy in the mockup here - it should be positioned so that it is clear that this will perform all actions when clicked (including any cloning instructions).
+ The cloning options have been gathered together, and should show all parts of the standard criteria
    + This should show what the current criteria is, and what we will be optionally cloning it to.
    + If no clone options are selected, then we are effectively just editing the current configuration (like in **phase 2**).
    + Some options should allow *Multiple* entries to be selected (like *Economy*) - others should be limited to a single option
        + This is indicated by the plurals - the key takeaway is that "Stream" is one at a time only.
    + The tool name cannot be changed, so does not have a dropdown selector.
        + We may want to revisit this later. Tools that use multiple unanchored models could be treated in the same way- i.e. we may want to push out a set of similar changes to a *FullEconomyValidator* and a *BHC Creator* - they work in much the same way in terms of inputs. But no need for this yet.
    + Only one of *Economy*, *Equity Asset* and *SubModel* can be chosen for cloning.
        + Otherwise, it leads to cross-joining all the combinations, which is unlikely what we want to happen anyway.
        + So if one of these selectors gets used, the other 2 should be disabled.
        
OK so far? Let's give some examples of what we want to happen for some realistic cases.

## CLONE of whole definition

To allow cloning of the current tool's TMT pattern, we need a way to indicate what we are changing in the current definition. In these examples, we want to select all model entries in the table, and no changes or removals will be made - we are taking everything as-is.  

### CLONE by stream

Take an existing definition to new single stream. In the example below, we will take a single copy of a RW credit tool in Stream "TestCandidate_A", and clone this across to a new stream "TestCandidate_C", using the original configuration.  

This could be represented in the proposed editor UI as follows:  

![](images/TMTEditor_Clone_ToNewStream.png)

So here, we are performing a single clone of an existing tool configuration by:

+ Selecting all models in the model table (we want everything here)
+ Select the new stream for the new tool run in the cloning options.
    + The other criteria are left alone - these will be the same as in the original case
+ We can now click the "Commit Changes" button to perfrom the clone
    + The user should be presented with a list of changes to TMT and asked if they wish to continue.  

> Should the editor redirect to the new configuration once it is complete? (or open in a new window?)

The operation should fail with a suitable warning if:

+ The configuration already exists.
+ Any of the outputs already exist as the output from another tool.

If this is successful, we should now expect to be able to see a configuration for this tool in the new stream with the following properties

+ The tool run should be in the new stream
+ The output models should change stream to the chosen stream by default.
+ Inputs will be identical to the original


### CLONE by economy

Take an existing definition and copy to new economies - multiple economies may be selected, and the current configuration **pattern** should be cloned to each new configuration.


This could be represented in the proposed editor UI as follows:  

![](images/TMTEditor_CloneToNewEcons.png)

So here, we are performing a clone of an existing tool configuration to multiple new tool runs by:

+ Selecting all models in the model table (we want everything here)
+ Select (multiple) economies that we want to clone the current pattern to.
    + The other criteria are left alone - these will be the same as in the original case
    + It should not matter if the current economy is selected here- we should recognise that one of the cases is just the original one and ignore it
+ We can now click the "Commit Changes" button to perfrom the clone
    + The user should be presented with a list of changes to TMT and asked if they wish to continue.  

> As this will create 2 new configurations, that may answer the question if we should redirect to the new copy or not - multiple new files kinda breaks this idea.

The operation should fail with a suitable warning if:

+ Any of the new configurations already exist.
+ Any of the newly created outputs already exist as the output from another tool.

If this is successful, we should now expect to be able to see a configuration for this tool in the same stream for each of the economies selected with the following properties

+ The tool run should be in the existing stream
+ The tool run should be in the newly chosen economies
+ The output models should be switched to match the new economy in each case
    + Assuming that the output used the original Economy as its economy setting (or for the *EconAssetOverride*)
    + The equivalent should happen if we are cloning a tool where *Equity Asset* is used in place of an economy
    + Likewise, if the change was to *SubModel*, then look there or in *SubModelOVerride*
+ Inputs *where there is a match on the economy* should be changed to use the new economy.
    + In this example case, that means *MarketDataBasis* and *MarketData.Bonds.Corporate*.
    + The other input in this example do not match their economy *with the original economy*, so should be unchanged


## CLONE using Stream/Economy/etc simultaneously

Combining both of the above cloning methods at the same time should be permitted. An example of this is as follows:

![](images/TMTEditor_Cloning_Example.png)

So here, we are performing a clone of an existing tool configuration to multiple new tool runs by:

+ Selecting all models in the model table (we want everything here)
+ Select (a single) new stream that we want to use for the new clone.
+ Select (multiple) economies that we want to clone the current pattern to.    
    + It **does** matter if the current economy is selected here - if we want that to be cloned to the new stream too, it should be included
+ Select a new **Curve Type** as well
    + This option should allow multiples.
    + This will only impact the tool runs and the *output* models - same rules apply as with *Stream*
    + The same rules for *Curve Type* should apply for 
+ We can now click the "Commit Changes" button to perfrom the clone
    + The user should be presented with a list of changes to TMT and asked if they wish to continue.  

The operation should fail with a suitable warning if:

+ Any of the new configurations already exist.
+ Any of the newly created outputs already exist as the output from another tool.

If this is successful, we should now expect to be able to see a configuration for this tool in the same stream for each of the (3) economies selected with the following properties

+ The tool run should be in the new stream
+ The tool run should be in the newly chosen economies
+ The tool run should be in the new *Curve Type*
+ The output models should be switched to match the new economy in each case
+ The output models should match the chosen *Curve Type* too.
    + No impact on the input models - we are choosing the least amount of enforced changes 
+ Inputs *where there is a match on the economy* should be changed to use the new economy.
    + In this example case, that means *MarketDataBasis* and *MarketData.Bonds.Corporate*.
    + The other input in this example do not match their economy *with the original economy*, so should be unchanged

## APPLY edit to multiple configurations

Rather than performing a full clone, we can use the TMT editor to make a change to a *single* tool run in the editor, and **broadcast** this change to other tools using the *cloning options*.  

A good example of where we want to do this is to take our cloned "Swap" calibrations from the simultaneous example above. On completion of that clone, we have created new valid configurations as far as TMT rules are concerned, but if we want these to behave like actual Swap calibrations, we need to make a few edits. As we want to make the same *type* of edit to all of these tool runs, we can:

+ Open up the TMT editor for *one* of these Tool Runs
+ Prepare the edit to the input models in the model table
    + So in this case, we are only selecting a subset of models with the select boxes, not all as in the cloning cases.
+ Use the "Cloning options" to define the other tools we want to apply the same change to.
    + Taking care that if we want to include this change to the current tool run, that has to be selected in the new config dropdowns that are edited.
+ When "Commit Changes" is clicked (and the change list is approved) the changes will be **Broadcast** to all tool runs as defined.


This would look like so:

![](images/TMTEditor_Clone_BroadcastChange.png)


Tricky to see in the mockup, but our changes are to edit the *MarketDataBasis* and add configuration for the previously missing *OutputDataBasis* model.  

On completion of the commit we would expect:

+ The two model changes to be applied to the original tool config
   + As "AUD" is in the list, which matches the original economy
+ The two model changes should be applied to the additional economies' tool config (CAD + JPY, with all other criteria matching the original).
    + These input model changes should apply using the same *pattern* as the change to the original tool config
        + i.e. these should still use the CAD/JPY economy - if these are AUD, then we have done something wrong
    + Likewise if we performed a REMOVE that we wanted to broadcast, this should find the equivalent model in the same economy as the tool run is under